In [1]:
import pandas as pd
import numpy as np
import multiprocessing
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import gc
from time import time
import datetime
from tqdm import tqdm_notebook
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.metrics import roc_auc_score
warnings.simplefilter('ignore')
sns.set()
%matplotlib inline

In [9]:
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold,TimeSeriesSplit, GroupKFold
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR

from sklearn.metrics import mean_absolute_error as sk_mean_absolute_error
from sklearn.metrics import roc_auc_score
import numpy as np
import pandas as pd

import eli5
from eli5.sklearn import PermutationImportance

class processutil:
    def _str2class(s):
        if s in globals() and isinstance(globals()[s], type):
                return globals()[s]
        if isinstance(eval(s), type):
            return eval(s)
        if callable(eval(s)):
            return eval(s)
        return None
    

In [3]:
folder_path = '../data/IEEE-CIS-Fraud-Detection/'

In [4]:
df_train = pd.read_pickle(f'{folder_path}/df_train2.gzde', compression='gzip').iloc[:10000,:]
df_test = pd.read_pickle(f'{folder_path}/df_test2.gzde', compression='gzip').iloc[:10000,:]

In [5]:
columns = df_test.columns.tolist()
columns.remove('TransactionID')
columns.remove('TransactionDT')

In [6]:
len(columns)

557

In [31]:
params = {
    'num_trees': 491,
   'depth': 9,
   'learning_rate': 0.05293979792364842,
#    'l2_leaf_reg': 78.065140245968,
#    'bagging_temperature': 0.9302786271852079,
#    'random_strength': 0.4247048326178351,
#    'random_state': 651,
    'logging_level': 'Silent'
         }

In [20]:
folds = TimeSeriesSplit(n_splits=5)

In [29]:
def process(folds, df_train,df_test, columns, is_output_feature_importance=1, verbose=0):

#     aucs = list()
    his = []
    training_start_time = time()
    df_feature_importances_i_list = []
    df_valid_pred = pd.DataFrame()
    df_test_pred = pd.DataFrame()
    if type(df_test) != type(None):
        df_test_pred['TransactionID'] = df_test['TransactionID']
    
    X,y = df_train.sort_values('TransactionDT')[columns], df_train.sort_values('TransactionDT')['isFraud']
    if type(df_test) != type(None):
        X_test = df_test.sort_values('TransactionDT')[columns]
        
        
#     cat_features_index = []
#     for idx, col in enumerate(columns):
#         if 'int' in str(df_train[col].dtype):
#             cat_features_index.append(idx)
        
    for fold, (trn_idx, test_idx) in enumerate(folds.split(X, y)):
        start_time = time()
        if verbose > 1:
            print('Training on fold {}'.format(fold + 1))
            
            
        clf = CatBoostClassifier(**params, eval_metric='AUC',use_best_model=True,random_seed=42)
        clf.fit(X.iloc[trn_idx].values, y.iloc[trn_idx].values,eval_set=(X.iloc[test_idx].values, y.iloc[test_idx].values))

        y_trn_pred = clf.predict(X.iloc[trn_idx].values)
        y_val_pred = clf.predict(X.iloc[test_idx].values)
        
        original_index = df_train['TransactionID'].values[test_idx]
        df_valid_pred_i = pd.DataFrame({'TransactionID': original_index, 'predict': y_val_pred, 'fold': np.zeros(y_val_pred.shape[0]) + fold})
        df_valid_pred = pd.concat([df_valid_pred, df_valid_pred_i], axis=0)
        
        y_test_pred = None
        if type(df_test)!=type(None):
            y_test_pred = clf.predict(X_test.values)
            df_test_pred_i = pd.DataFrame({fold: y_test_pred})
            df_test_pred = pd.concat([df_test_pred, df_test_pred_i], axis=1)
        
        trn_auc = roc_auc_score(y.iloc[trn_idx].values, y_trn_pred)
        val_auc = roc_auc_score(y.iloc[test_idx].values, y_val_pred)
        
        his.append({'val_auc':val_auc, 'trn_auc':trn_auc, 'y_val_pred':y_val_pred, 'y_test_pred':y_test_pred, 'test_idx':test_idx})
        
        if is_output_feature_importance:
            perm = PermutationImportance(clf, random_state=42).fit(X.iloc[test_idx].values, y.iloc[test_idx].values)
            df_feature_importances_i2 = eli5.explain_weights_dfs(perm, feature_names=columns, top=len(columns))['feature_importances']
            df_feature_importances_i2 = df_feature_importances_i2.sort_values(by=['feature'])
            df_feature_importances_i2 = df_feature_importances_i2.reset_index(drop=True)
            df_feature_importances_i_list.append(df_feature_importances_i2)
        
#         aucs.append(clf.best_score['valid_1']['auc'])
#         his.append({'val_auc':val_auc, 'trn_auc':trn_auc, 'y_val_pred':y_val_pred, 'y_test_pred':y_test_pred, 'test_idx':test_idx})
        if verbose > 0:
            print('Fold {} finished in {}'.format(fold + 1, str(datetime.timedelta(seconds=time() - start_time))))
    
    his = pd.DataFrame(his)
    
    df_feature_importances = None
    if is_output_feature_importance:
        df_feature_importances = df_feature_importances_i_list[0]
        for idx, df_feature_importances_i in enumerate(df_feature_importances_i_list[1:]):
            df_feature_importances = pd.merge(df_feature_importances, df_feature_importances_i, on='feature', suffixes=('', idx + 1))
            
    df_valid_pred = df_valid_pred.sort_values(by=['TransactionID'])
    df_valid_pred = df_valid_pred.reset_index(drop=True)

    if type(df_test) != type(None):
        df_test_pred = df_test_pred.sort_values(by=['TransactionID'])
        df_test_pred = df_test_pred.reset_index(drop=True)
    
    if verbose > 0:
        print('-' * 30)
        print('Training has finished.')
        print('Total training time is {}'.format(str(datetime.timedelta(seconds=time() - training_start_time))))
        print('Mean AUC:', his.val_auc.mean(), his.trn_auc.mean())
        print('-' * 30)
    return his, df_feature_importances, df_valid_pred, df_test_pred, his.val_auc.mean()

In [ ]:
his, df_feature_importances, df_valid_pred, df_test_pred, val_metric = process(folds, df_train,df_test, columns, is_output_feature_importance=1, verbose=1)

In [ ]:
his

In [35]:
def sort_feature_importances(df_feature_importances, key='average_permutation_weight'):
    df_feature_importances['average_permutation_weight'] = df_feature_importances[
        [col for col in df_feature_importances.columns.tolist() if ('weight' in col) & ('model' not in col)]].mean(
        axis=1)
    df_feature_importances = df_feature_importances.sort_values(by=[key], ascending=False)
    sorted_columns = df_feature_importances.feature.tolist()
    return sorted_columns

In [40]:
sorted_columns = sort_feature_importances(df_feature_importances)

In [47]:
def add_feature_obo(folds, df_train, columns, max_col = 50):
    selected_columns = [columns[0]]
    if type(max_col) == type(None):
        max_col = len(columns)
    best_val_metric = 0
    for col in columns[1:]:
        his, df_feature_importances, df_valid_pred, df_test_pred, val_metric = process(folds, df_train,None, selected_columns + [col], is_output_feature_importance=0, verbose=0)
        if val_metric > best_val_metric:
            selected_columns = selected_columns + [col]
            best_val_metric = val_metric
        if len(selected_columns) > max_col:
            break
    return selected_columns

In [ ]:
select_columns = add_feature_obo(folds, df_train, sorted_columns, max_col = 50)

Training until validation scores don't improve for 500 rounds.
Early stopping, best iteration is:
[378]	training's auc: 0.880269	valid_1's auc: 0.724129
Training until validation scores don't improve for 500 rounds.
[1000]	training's auc: 0.893908	valid_1's auc: 0.735808
Early stopping, best iteration is:
[836]	training's auc: 0.886029	valid_1's auc: 0.745267
Training until validation scores don't improve for 500 rounds.
[1000]	training's auc: 0.880981	valid_1's auc: 0.753178
[2000]	training's auc: 0.899852	valid_1's auc: 0.775654
[3000]	training's auc: 0.913536	valid_1's auc: 0.785986
Early stopping, best iteration is:
[2966]	training's auc: 0.913302	valid_1's auc: 0.78836
Training until validation scores don't improve for 500 rounds.
[1000]	training's auc: 0.889475	valid_1's auc: 0.76656
[2000]	training's auc: 0.905053	valid_1's auc: 0.778868
[3000]	training's auc: 0.912893	valid_1's auc: 0.785172
[4000]	training's auc: 0.916957	valid_1's auc: 0.788756
[5000]	training's auc: 0.920297

Early stopping, best iteration is:
[9]	training's auc: 0.867406	valid_1's auc: 0.861463
Training until validation scores don't improve for 500 rounds.
Early stopping, best iteration is:
[2]	training's auc: 0.862575	valid_1's auc: 0.819471
Training until validation scores don't improve for 500 rounds.
Early stopping, best iteration is:
[3]	training's auc: 0.873442	valid_1's auc: 0.816226
Training until validation scores don't improve for 500 rounds.
Early stopping, best iteration is:
[23]	training's auc: 0.898567	valid_1's auc: 0.749956
Training until validation scores don't improve for 500 rounds.
Early stopping, best iteration is:
[5]	training's auc: 0.845445	valid_1's auc: 0.831455
Training until validation scores don't improve for 500 rounds.
Early stopping, best iteration is:
[38]	training's auc: 0.871482	valid_1's auc: 0.854559
Training until validation scores don't improve for 500 rounds.
[1000]	training's auc: 0.940333	valid_1's auc: 0.82851
[2000]	training's auc: 0.96001	valid_

[1000]	training's auc: 0.994303	valid_1's auc: 0.879337
Early stopping, best iteration is:
[1124]	training's auc: 0.995854	valid_1's auc: 0.880191
Training until validation scores don't improve for 500 rounds.
[1000]	training's auc: 0.995971	valid_1's auc: 0.861233
[2000]	training's auc: 0.999527	valid_1's auc: 0.862861
Early stopping, best iteration is:
[1635]	training's auc: 0.999054	valid_1's auc: 0.863711
Training until validation scores don't improve for 500 rounds.
[1000]	training's auc: 0.995611	valid_1's auc: 0.84805
Early stopping, best iteration is:
[1252]	training's auc: 0.997736	valid_1's auc: 0.848714
Training until validation scores don't improve for 500 rounds.
Early stopping, best iteration is:
[6]	training's auc: 0.915089	valid_1's auc: 0.755805
Training until validation scores don't improve for 500 rounds.
Early stopping, best iteration is:
[73]	training's auc: 0.921677	valid_1's auc: 0.834101
Training until validation scores don't improve for 500 rounds.
[1000]	train

In [ ]:
select_columns